# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task,
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# !gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
# !cp ./data.zip /content/drive/MyDrive/data.zip

!cp /content/drive/MyDrive/data.zip ./
!unzip data.zip
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  data.zip
replace timit_11/train_11.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace timit_11/test_11.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace timit_11/train_label_11.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
data.zip  drive  model0.ckpt  model1.ckpt  sample_data	timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None): #x:feature y:label
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0 #0 train all data

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1229932, 429)
Size of validation set: (0, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 128

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

167

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [ ]:
import torch
import torch.nn as nn

class Classifier1(nn.Module):
    def __init__(self):
        super(Classifier1, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(429, 2048),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(2048,1024),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(1024, 728),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(728, 512),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),


            nn.Linear(128, 39)

        )

    def forward(self, x):

      return self.net(x).squeeze(1)

class Classifier2(nn.Module):
    def __init__(self):
        super(Classifier2, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(429, 2000),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(2000,2000),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(2000, 80),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(80, 2000),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(2000, 39)
        )

    def forward(self, x):

      return self.net(x).squeeze(1)

class Classifier3(nn.Module):
    def __init__(self):
        super(Classifier3, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(429, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(1024,2048),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(2048, 728),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(728, 512),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(128, 39)

        )

    def forward(self, x):

      return self.net(x).squeeze(1)


## Training

In [ ]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [ ]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [ ]:
# get device
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch =  50              # number of training epoch
learning_rate = 0.0001       # learning rate

DEVICE: cuda


In [ ]:
# start training
seeds=[7987, 886, 4758, 56879, 49, 76577]
# fix random seed for reproducibility
for j in range(3) :

  same_seeds(seeds[j])

  # the path where checkpoint saved
  model_path = f'./model{j}.ckpt'
  model = Classifier2().to(device)

  # create model, define a loss function, and optimizer
  # if j%3 == 1 :
  #   model = Classifier1().to(device)
  # elif j%3 == 2 :
  #
  # else :
  #   model = Classifier3().to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

  best_acc = 0.0
  loss_record = {'train': [], 'dev': []}
  print("{}th model".format(j))
  for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

      # training

    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward()
        optimizer.step()

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()


    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels)
                _, val_pred = torch.max(outputs, 1)

                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()


            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))
            loss_record['train'].append(train_acc/len(train_set))
            loss_record['dev'].append(val_acc/len(val_set))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

  # if not validating, save the last epoch
  if len(val_set) == 0:
      torch.save(model.state_dict(), model_path)
      print('saving model at last epoch')

  print("{}th model finished".format(j))


0th model
[001/050] Train Acc: 0.540486 Loss: 1.537680 | Val Acc: 0.665471 loss: 1.114413
saving model with acc 0.665
[002/050] Train Acc: 0.625342 Loss: 1.234631 | Val Acc: 0.690383 loss: 1.009117
saving model with acc 0.690
[003/050] Train Acc: 0.652341 Loss: 1.134631 | Val Acc: 0.708116 loss: 0.946599
saving model with acc 0.708
[004/050] Train Acc: 0.668678 Loss: 1.073739 | Val Acc: 0.716084 loss: 0.920045
saving model with acc 0.716
[005/050] Train Acc: 0.680220 Loss: 1.030023 | Val Acc: 0.719328 loss: 0.902092
saving model with acc 0.719
[006/050] Train Acc: 0.689006 Loss: 0.998409 | Val Acc: 0.723800 loss: 0.889491
saving model with acc 0.724
[007/050] Train Acc: 0.696201 Loss: 0.970285 | Val Acc: 0.730458 loss: 0.868814
saving model with acc 0.730
[008/050] Train Acc: 0.702875 Loss: 0.947257 | Val Acc: 0.732141 loss: 0.859313
saving model with acc 0.732
[009/050] Train Acc: 0.708558 Loss: 0.927132 | Val Acc: 0.733849 loss: 0.853472
saving model with acc 0.734
[010/050] Train Ac

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

def plot_learning_curve(loss_record, title=''):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    total_steps = len(loss_record['train'])
    x_1 = range(total_steps)
    x_2 = x_1[::len(loss_record['train']) // len(loss_record['dev'])]
    figure(figsize=(6, 4))

    plt.plot(x_1, loss_record['train'], c='tab:red', label='train')
    plt.plot(x_2, loss_record['dev'], c='tab:cyan', label='dev')
    plt.ylim(0.5, 1.)
    plt.xlabel('Training steps')
    plt.ylabel('MSE loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()

In [ ]:
plot_learning_curve(loss_record, title='deep model')

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier1().to(device)
possibility = np.zeros((451552,39))

for i in range(9):
  print(i)
  j=i
  if j < 3 :
    model = Classifier1().to(device)

  elif j < 6 :
    model = Classifier2().to(device)

  else :
    model = Classifier3().to(device)

  model.load_state_dict(torch.load(f'./model{i}.ckpt'))
  model.eval() # set the model to evaluation mode

  with torch.no_grad():
      for i, data in enumerate(test_loader):
          inputs = data
          inputs = inputs.to(device)
          outputs = model(inputs).cpu().numpy()
          p_bar = np.zeros(outputs.shape)
          for ind, j in enumerate(np.argmax(outputs, axis=1)):
            p_bar[ind][j] = 1

          # p_bar[np.arange(128),np.argmax(outputs, axis=1).reshape(-1,1)] = 1



          if i == 0 :
            # print("out",outputs, outputs.shape)
            # print(np.argmax(outputs, axis=1), np.argmax(outputs, axis=1).shape)
            # print("p_bar",p_bar, p_bar.shape)
            p = p_bar
            # print(np.shape(outputs))
          else :
            p = np.append(p, p_bar, axis =0)
            # print("outputs",np.shape(outputs))
            # print("p",np.shape(p))


  # print(np.shape(p))
  possibility += p
  # print(possibility)

predict = np.argmax(possibility, axis =1)

# _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

#           for y in test_pred.cpu().numpy():
#               predict.append(y)


0
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
1
[[0. 0. 0. ... 2. 0. 0.]
 [0. 0. 0. ... 2. 0. 0.]
 [0. 0. 0. ... 2. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
2
[[0. 0. 0. ... 3. 0. 0.]
 [0. 0. 0. ... 3. 0. 0.]
 [0. 0. 0. ... 3. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
3
[[0. 0. 0. ... 4. 0. 0.]
 [0. 0. 0. ... 4. 0. 0.]
 [0. 0. 0. ... 4. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
4
[[0. 0. 0. ... 5. 0. 0.]
 [0. 0. 0. ... 5. 0. 0.]
 [0. 0. 0. ... 5. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
5
[[0. 0. 0. ... 6. 0. 0.]
 [0. 0. 0. ... 6. 0. 0.]
 [0. 0. 0. ... 6. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
6
[[0. 0. 0. ... 7. 0. 0.]
 [0. 0. 0. ... 7. 0. 0.]


Make prediction.

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))